In [2]:
!pip install --quiet mrjob==0.7.4

     |████████████████████████████████| 439 kB 6.5 MB/s 


In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


#**ID3(Sequential)**

In [4]:
import csv
import math
import copy

class DataSet:
    headers = []
    rows = []

    def get_width(self):
        return self.headers.__len__()

    def get_size(self):
        return self.rows.__len__()

    def get_class_index(self):
        return self.headers.__len__() - 1


class Tree(object):
    def __init__(self, name='root', children=None):
        self.name = name
        self.value = ''
        self.children = []
        if children is not None:
            for child in children:
                self.add_child(child)

    def __repr__(self):
        return self.name

    def add_child(self, node):
        assert isinstance(node, Tree)
        self.children.append(node)

    def is_pure(self):
        return self.children.__len__() == 0


def read_data(filename='') -> DataSet:
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        headers = []
        rows = []
        for row in csv_reader:
            if line_count == 0:
                headers += row
                line_count += 1
            else:
                rows.append(row)
                line_count += 1

    data = DataSet()
    data.headers = headers
    data.rows = rows

    return data


def remove_attr(dataset: DataSet, index: int) -> DataSet:
    new_headers = dataset.headers[:index] + dataset.headers[index + 1:]
    new_rows = []
    for row in dataset.rows:
        new_rows.append(row[:index] + row[index + 1:])

    new_dataset = DataSet()
    new_dataset.headers = new_headers
    new_dataset.rows = new_rows
    return new_dataset


def extract_feature(dataset: DataSet, col_index: int, future='') -> DataSet:
    new_rows = []
    for row in dataset.rows:
        if row[col_index] == future:
            new_rows.append(copy.copy(row))

    new_dataset = DataSet()
    new_dataset.headers = copy.copy(dataset.headers)
    new_dataset.rows = new_rows
    return new_dataset




def calc_attribute_info(dataset: DataSet, col_index: int):
    diff_features = []
    info = 0
    for row in dataset.rows:
        if row[col_index] not in diff_features:
            diff_features.append(row[col_index])
            subset = extract_feature(dataset, col_index, row[col_index])
            info += calc_info(subset) * (subset.get_size() / dataset.get_size())
            # print_dataset(subset)

    # print(info , diff_features)


    return {'info': info, 'features': diff_features}


def calc_info(dataset: DataSet) -> float:
    rows = dataset.rows
    class1label = rows[0][dataset.get_class_index()]
    class1count = 0
    class2count = 0

    for i in range(dataset.get_size()):
        if rows[i][dataset.get_class_index()] == class1label:
            class1count += 1
        else:
            class2count += 1

    pclass1 = class1count / dataset.get_size()
    pclass2 = class2count / dataset.get_size()

    if pclass2 == 0:
        return 0.0

    info = (-pclass1 * math.log(pclass1, 2)) + (-pclass2 * math.log(pclass2, 2))
    return info


def ID3(dataset: DataSet, org_dataset_info: float,root: Tree):
    best_gain = 0.0
    best_col_index = 0
    features = []
    for col_index in range(dataset.get_width() - 1):  # ignore class attr
        res = calc_attribute_info(dataset, col_index)
        gain = org_dataset_info - res['info']
        if gain > best_gain:
            best_gain = gain
            best_col_index = col_index
            features = res['features']

    for feature in features:
        new_node = Tree()
        new_node.name = dataset.headers[best_col_index]
        new_node.value = feature

        new_subset = extract_feature(dataset,best_col_index,feature)
        new_subset = remove_attr(new_subset, best_col_index)
        info = calc_info(new_subset)
        if info == 0:
            pure_node = Tree(new_subset.headers[new_subset.get_class_index()])
            pure_node.value = new_subset.rows[0][new_subset.get_class_index()]
            new_node.add_child(pure_node)
            pure_node = None
        else:
            ID3(new_subset, org_dataset_info,new_node)  # call function recursive

        root.add_child(new_node)


def print_dataset(dataset: DataSet):
    print()
    print(dataset.headers)
    for row in dataset.rows:
        print(row)


def print_tree(root: Tree):

    if root.is_pure():
        print(' THAN ' + root.name + ' ' + root.value)
    else:
        if root.name != 'root':
            print(' IF ' + root.name + ' IS ' + root.value, end='')
            print(' AND ', end='')
        for child in root.children:
            print_tree(child)


def pprint_tree(node: Tree, file=None, _prefix="", _last=True):
    print(_prefix, "`- " if _last else "|- ", node.name +' - '+node.value, sep="", file=file)
    _prefix += "   " if _last else "|  "
    child_count = len(node.children)
    for i, child in enumerate(node.children):
        _last = i == (child_count - 1)
        pprint_tree(child, file, _prefix, _last)


def main():

    data = read_data('/content/gdrive/My Drive/IT488/Data.csv')
    info = calc_info(data)
    root = Tree()
    ID3(data, info,root)
    pprint_tree(root)



main()

`- root - 
   |- Discount - Yes
   |  |- Holiday - No
   |  |  `- Purchase - Yes
   |  `- Holiday - yes
   |     |- Free Delivery - Yes
   |     `- Free Delivery - No
   |        `- Purchase - Yes
   `- Discount - No


#**C4.5(Sequential)**

In [13]:
import csv
import math
import copy
import time


class DataSet:
    headers = []
    rows = []

    def get_width(self):
        return self.headers.__len__()

    def get_size(self):
        return self.rows.__len__()

    def get_class_index(self):
        return self.headers.__len__() - 1


class Tree(object):
    def __init__(self, name='root', children=None):
        self.name = name
        self.value = ''
        self.children = []
        if children is not None:
            for child in children:
                self.add_child(child)

    def __repr__(self):
        return self.name

    def add_child(self, node):
        assert isinstance(node, Tree)
        self.children.append(node)

    def is_pure(self):
        return self.children.__len__() == 0




def read_data(filename='') -> DataSet:
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        headers = []
        rows = []
        for row in csv_reader:
            if line_count == 0:
                headers += row
                line_count += 1
            else:
                rows.append(row)
                line_count += 1

    data = DataSet()
    data.headers = headers
    data.rows = rows

    return data


def remove_attr(dataset: DataSet, index: int) -> DataSet:
    new_headers = dataset.headers[:index] + dataset.headers[index + 1:]
    new_rows = []
    for row in dataset.rows:
        new_rows.append(row[:index] + row[index + 1:])

    new_dataset = DataSet()
    new_dataset.headers = new_headers
    new_dataset.rows = new_rows
    return new_dataset


def extract_feature(dataset: DataSet, col_index: int, future='') -> DataSet:
    new_rows = []
    for row in dataset.rows:
        if row[col_index] == future:
            new_rows.append(copy.copy(row))

    new_dataset = DataSet()
    new_dataset.headers = copy.copy(dataset.headers)
    new_dataset.rows = new_rows
    return new_dataset


def calc_attribute_info(dataset: DataSet , col_index: int):
    diff_features = []
    info = 0
    splitinfo=0


    for row in dataset.rows:
        if row[col_index] not in diff_features:
            diff_features.append(row[col_index])
            subset = extract_feature(dataset, col_index, row[col_index])

            if(subset.get_size() == dataset.get_size()):
                info=1
                splitinfo=0.0000000000000000000001        #taking tends to 0.
                return {'info': info, 'features': diff_features, 'splitinfo' : splitinfo}

            info += calc_info(subset) * (subset.get_size() / dataset.get_size())
            splitinfo += -1*math.log(subset.get_size() / dataset.get_size()) * (subset.get_size() / dataset.get_size())
            # print_dataset(subset)


    return {'info': info, 'features': diff_features, 'splitinfo' : splitinfo}


def calc_info(dataset: DataSet) -> float:
    rows = dataset.rows
    class1label = rows[0][dataset.get_class_index()]
    class1count = 0
    class2count = 0

    for i in range(dataset.get_size()):
        if rows[i][dataset.get_class_index()] == class1label:
            class1count += 1
        else:
            class2count += 1

    pclass1 = class1count / dataset.get_size()
    pclass2 = class2count / dataset.get_size()

    if pclass2 == 0:
        return 0.0

    info = (-pclass1 * math.log(pclass1, 2)) + (-pclass2 * math.log(pclass2, 2))
    return info


def c45(dataset: DataSet, org_dataset_info: float,root: Tree):
    best_gain = 0.0
    best_col_index = 0
    features = []
    for col_index in range(dataset.get_width() - 1):  # ignore class attr
        res = calc_attribute_info(dataset, col_index)   ########here we need to change

        gain = org_dataset_info - res['info']

        gain=gain/res['splitinfo']

        if gain > best_gain:
            best_gain = gain
            best_col_index = col_index
            features = res['features']

    for feature in features:
        new_node = Tree()
        new_node.name = dataset.headers[best_col_index]
        new_node.value = feature

        new_subset = extract_feature(dataset,best_col_index,feature)
        new_subset = remove_attr(new_subset, best_col_index)
        info = calc_info(new_subset)
        if info == 0:
            pure_node = Tree(new_subset.headers[new_subset.get_class_index()])
            pure_node.value = new_subset.rows[0][new_subset.get_class_index()]
            new_node.add_child(pure_node)
            pure_node = None
        else:
            c45(new_subset, org_dataset_info,new_node)  # call function recursive

        root.add_child(new_node)


def print_dataset(dataset: DataSet):
    print(dataset.headers)
    for row in dataset.rows:
        print(row)


def print_tree(root: Tree):
    if root.is_pure():
        print(' THAN ' + root.name + ' ' + root.value)
    else:
        if root.name != 'root':
            print(' IF ' + root.name + ' IS ' + root.value, end='')
            print(' AND ', end='')
        for child in root.children:
            print_tree(child)


def pprint_tree(node: Tree, file=None, _prefix="", _last=True):
    print(_prefix, "`- " if _last else "|- ", node.name +' - '+node.value, sep="", file=file)
    _prefix += "   " if _last else "|  "
    child_count = len(node.children)
    for i, child in enumerate(node.children):
        _last = i == (child_count - 1)
        pprint_tree(child, file, _prefix, _last)


def main():

    data = read_data('/content/gdrive/My Drive/IT488/Data.csv')
    info = calc_info(data)
    root = Tree()

    st_time = time.time()
        
    c45(data, info,root)

    print_tree(root)

    pprint_tree(root)

    en_time = time.time()

    print("Time taken by C4.5 Sequential : ",en_time-st_time)


main()

 IF Free Delivery IS Yes AND  IF Discount IS Yes AND  IF Holiday IS No AND  THAN Purchase Yes
 THAN Holiday yes
 IF Discount IS No AND  IF Holiday IS yes AND  THAN Purchase Yes
 IF Holiday IS No AND  THAN Purchase No
 IF Free Delivery IS No AND  THAN Discount No
 IF Discount IS Yes AND  THAN Purchase Yes
`- root - 
   |- Free Delivery - Yes
   |  |- Discount - Yes
   |  |  |- Holiday - No
   |  |  |  `- Purchase - Yes
   |  |  `- Holiday - yes
   |  `- Discount - No
   |     |- Holiday - yes
   |     |  `- Purchase - Yes
   |     `- Holiday - No
   |        `- Purchase - No
   `- Free Delivery - No
      |- Discount - No
      `- Discount - Yes
         `- Purchase - Yes
Time taken by C4.5 Sequential :  0.016287565231323242


#****C4.5(Parallel using Map-Reduce)****

In [9]:
%%file temp.py

from mrjob.job import MRJob

class Dtree(MRJob):

    def mapper(self,key,value): 
        value = value.strip().split(',')
        if(value[-1].lower() == "yes"):
            yield value[0],[1,1]
        else:
            yield value[0],[1,0]

    def reducer(self,key,value):
        total = 0
        totaly = 0
        totaln = 0

        for val in value:
            total += 1
            if(val[1] == 1):
                totaly += 1
            else:
                totaln += 1
        
        yield key , [total,totaln,totaly]

if __name__ == '__main__':
    Dtree.run()

Writing temp.py


In [12]:
import csv
import math
import copy
from mrjob.job import MRJob
from temp import Dtree

class DataSet:
    headers = []
    rows = []

    def get_width(self):
        return self.headers.__len__()

    def get_size(self):
        return self.rows.__len__()

    def get_class_index(self):
        return self.headers.__len__() - 1



class Tree(object):
    def __init__(self, name='root', children=None):
        self.name = name
        self.value = ''
        self.children = []
        if children is not None:
            for child in children:
                self.add_child(child)

    def __repr__(self):
        return self.name

    def add_child(self, node):
        assert isinstance(node, Tree)
        self.children.append(node)

    def is_pure(self):
        return self.children.__len__() == 0


def read_data(filename='') -> DataSet:
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        headers = []
        rows = []
        for row in csv_reader:
            if line_count == 0:
                headers += row
                line_count += 1
            else:
                rows.append(row)
                line_count += 1

    data = DataSet()
    data.headers = headers
    data.rows = rows

    return data


def remove_attr(dataset: DataSet, index: int) -> DataSet:
    new_headers = dataset.headers[:index] + dataset.headers[index + 1:]
    new_rows = []
    for row in dataset.rows:
        new_rows.append(row[:index] + row[index + 1:])

    new_dataset = DataSet()
    new_dataset.headers = new_headers
    new_dataset.rows = new_rows
    return new_dataset


def extract_feature(dataset: DataSet, col_index: int, future='') -> DataSet:
    new_rows = []
    for row in dataset.rows:
        if row[col_index] == future:
            new_rows.append(copy.copy(row))

    new_dataset = DataSet()
    new_dataset.headers = copy.copy(dataset.headers)
    new_dataset.rows = new_rows
    return new_dataset


def calc_attribute_info(dataset: DataSet , col_index: int):
    
    datasize = dataset.get_size()
    
    List = list()

    tempinp = "/content/gdrive/My Drive/IT488/inp.csv"

    f = open(tempinp, "w") 

    for line in dataset.rows:
        datasize += 1
        L = str(line[col_index]) + ',' + str(line[-1])
        f.writelines(L + "\n")
    f.close()

    inputFile = "/content/gdrive/My Drive/IT488/inp.csv"

    # outFile = tempFile

    mr_job = Dtree(args=[inputFile])
    
    with mr_job.make_runner() as runner:
        runner.run()

        for key, value in mr_job.parse_output(runner.cat_output()):
            List.append([key,value])

  

    info = 0.0
    diff_features = list()
    splitinfo = 0.0

    for val in List:
        total = val[1][0]
        totaly = val[1][1]
        totaln = val[1][2]

        if(total == datasize):
            info=1
            splitinfo=0.0000000000000000000001        #taking tends to 0.
            return {'info': info, 'features': diff_features, 'splitinfo' : splitinfo}

        res = 0.0
        if(totaln != 0 and totaly != 0):
            res = 1.0*(total/datasize)
            py = 1.0*(totaly/total) 
            pn = 1.0*(totaln/total)
            Ans = (-py * math.log(py, 2)) + (-pn * math.log(pn, 2))
            res = res*Ans

        info += res
        splitinfo += (-(total/datasize)*math.log((total/datasize),2))
        diff_features.append(val[0])

    # print(info , diff_features)

    # return {'info': info, 'features': diff_features }

    return {'info': info, 'features': diff_features, 'splitinfo' : splitinfo}


def calc_info(dataset: DataSet) -> float:
    rows = dataset.rows
    class1label = rows[0][dataset.get_class_index()]
    class1count = 0
    class2count = 0

    for i in range(dataset.get_size()):
        if rows[i][dataset.get_class_index()] == class1label:
            class1count += 1
        else:
            class2count += 1

    pclass1 = class1count / dataset.get_size()
    pclass2 = class2count / dataset.get_size()

    if pclass2 == 0:
        return 0.0

    info = (-pclass1 * math.log(pclass1, 2)) + (-pclass2 * math.log(pclass2, 2))
    return info


def c45(dataset: DataSet, org_dataset_info: float,root: Tree):
    best_gain = 0.0
    best_col_index = 0
    features = []
    for col_index in range(dataset.get_width() - 1):  # ignore class attr
        res = calc_attribute_info(dataset, col_index)   ########here we need to change

        #mapper_()

        gain = org_dataset_info - res['info']

        gain=gain/res['splitinfo'] #If we comment this line then code will be for ID3 Map-Reduce algorithm.

        if gain > best_gain:
            best_gain = gain
            best_col_index = col_index
            features = res['features']

    for feature in features:
        new_node = Tree()
        new_node.name = dataset.headers[best_col_index]
        new_node.value = feature

        new_subset = extract_feature(dataset,best_col_index,feature)
        new_subset = remove_attr(new_subset, best_col_index)
        info = calc_info(new_subset)
        if info == 0:
            pure_node = Tree(new_subset.headers[new_subset.get_class_index()])
            pure_node.value = new_subset.rows[0][new_subset.get_class_index()]
            new_node.add_child(pure_node)
            pure_node = None
        else:
            c45(new_subset, org_dataset_info,new_node)  # call function recursive

        root.add_child(new_node)


def print_dataset(dataset: DataSet):
    print()
    print(dataset.headers)
    for row in dataset.rows:
        print(row)


def print_tree(root: Tree):

    if root.is_pure():
        print(' THAN ' + root.name + ' ' + root.value)
    else:
        if root.name != 'root':
            print(' IF ' + root.name + ' IS ' + root.value, end='')
            print(' AND ', end='')
        for child in root.children:
            print_tree(child)


def pprint_tree(node: Tree, file=None, _prefix="", _last=True):
    print(_prefix, "`- " if _last else "|- ", node.name +' - '+node.value, sep="", file=file)
    _prefix += "   " if _last else "|  "
    child_count = len(node.children)
    for i, child in enumerate(node.children):
        _last = i == (child_count - 1)
        pprint_tree(child, file, _prefix, _last)


def main():

    data = read_data('/content/gdrive/My Drive/IT488/Data100.csv')
    info = calc_info(data)
    root = Tree()

    st_time = time.time()

    c45(data, info,root)

    en_time = time.time()

    print("Time taken by C4.5 with Map-Reduce: ",en_time-st_time)

    pprint_tree(root)



main()

Time taken by C4.5 with Map-Reduce:  0.8634135723114014
`- root - 
   |- Free Delivery - No
   |  |- Discount - Yes
   |  |  `- Purchase - Yes
   |  `- Discount - No
   |     |- Holiday - No
   |     |  `- Purchase - No
   |     `- Holiday - yes
   `- Free Delivery - Yes
      |- Discount - No
      |  |- Holiday - No
      |  |  `- Purchase - No
      |  `- Holiday - yes
      |     `- Purchase - Yes
      `- Discount - Yes
         |- Holiday - yes
         `- Holiday - No
            `- Purchase - Yes
